In [1]:
# 저장한 모델 불러오기
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

from transformers import AutoModel, AutoTokenizer, BertTokenizer
MODEL_NAME = "beomi/KcELECTRA-base-v2022"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

tag2id = {'O': 0, 'B-PER': 1, 'I-PER': 2}
unique_tags={'B-PER', 'I-PER', 'O'}
id2tag = {0: 'O', 1: 'B-PER', 2: 'I-PER'}
pad_token_id = tokenizer.pad_token_id # 0
cls_token_id = tokenizer.cls_token_id # 101
sep_token_id = tokenizer.sep_token_id # 102
pad_token_label_id = tag2id['O']    # tag2id['O']
cls_token_label_id = tag2id['O']
sep_token_label_id = tag2id['O']

C:\Users\LG\anaconda3\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\LG\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [2]:
# model_path = '../model/kcelectra_base_ner_law'
# model_path = '../model/kcelectra_base_new'
model = AutoModelForTokenClassification.from_pretrained(model_path, num_labels=len(unique_tags))
model.to(device)

ElectraForTokenClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(54343, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0-11): 12 x ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): Laye

In [3]:
def ner_tokenizer(sent, max_seq_length):
    pre_syllable = "_"
    input_ids = [pad_token_id] * (max_seq_length - 1)
    attention_mask = [0] * (max_seq_length - 1)
    token_type_ids = [0] * max_seq_length
    sent = sent[:max_seq_length-2]

    for i, syllable in enumerate(sent):
        if syllable == '_':
            pre_syllable = syllable
        if pre_syllable != "_":
            syllable = '##' + syllable  # 중간 음절에는 모두 prefix를 붙입니다.
            # 우리가 구성한 학습 데이터도 이렇게 구성되었기 때문이라고 함.
            # 이순신은 조선 -> [이, ##순, ##신, ##은, 조, ##선]
        pre_syllable = syllable

        input_ids[i] = (tokenizer.convert_tokens_to_ids(syllable))
        attention_mask[i] = 1

    input_ids = [cls_token_id] + input_ids
    input_ids[len(sent)+1] = sep_token_id
    attention_mask = [1] + attention_mask
    attention_mask[len(sent)+1] = 1
    return {"input_ids":input_ids,
            "attention_mask":attention_mask,
            "token_type_ids":token_type_ids}

In [4]:
def ner_inference(text):
    model.eval()
    text = text.replace(' ', '_')

    predictions, true_labels = [], []
    law_list = []  # 법을 담을 리스트

    tokenized_sent = ner_tokenizer(text, len(text) + 2)
    input_ids = torch.tensor(tokenized_sent['input_ids']).unsqueeze(0).to(device)
    attention_mask = torch.tensor(tokenized_sent['attention_mask']).unsqueeze(0).to(device)
    token_type_ids = torch.tensor(tokenized_sent['token_type_ids']).unsqueeze(0).to(device)

    with torch.no_grad():
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids)

    logits = outputs['logits']
    logits = logits.detach().cpu().numpy()
    label_ids = token_type_ids.cpu().numpy()

    predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
    true_labels.append(label_ids)

    pred_tags = [list(tag2id.values())[p_i] for p in predictions for p_i in p]
    tokenized_text = tokenizer.convert_ids_to_tokens(tokenized_sent['input_ids'])

#     print('{}\t{}'.format("TOKEN", "TAG"))
#     print("===========")
    for token, tag in zip(tokenized_text, pred_tags):
#         print("{:^5}\t{:^5}".format(token, tag))
        if tag == 1 or tag == 2:
            law_list.append(token)
    return law_list

In [5]:
text2 = '''홍익표 더불어민주당 원내대표는 31일 윤석열 대통령이 재의요구권(거부권)을 행사한 '이태원 참사 특별법'과 관련해 국민의힘이 기존 입장을 고수할 경우 2월 국회에서 재의결을 추진하겠다고 밝혔다.

홍 원내대표는 이날 오전 라디오 '김종배의 시선집중'에 출연해 "한번 협상은 해보겠다만 여당이 기존의 입장에서 변화가 없다면 사실상 재협상의 실질적 진전이 있기는 어려울 것"이라고 말했다.

'''

In [6]:
laws2 = ner_inference(text2)

In [7]:
laws2

['##이', '##태', '##원', '_', '참', '##사', '_', '특', '##별', '##법']

In [8]:
text1 = '''공정거래위원회가 소수 거대 플랫폼의 독과점을 막겠다는 취지의 '플랫폼 공정경쟁 촉진법' 제정을 추진하는 데 대해 학계와 스타트업, 소비자 단체가 일제히 반대의 목소리를 냈다.
31일 서울 종로구 한국프레스센터에서 온라인 플랫폼 규제의 쟁점 진단을 주제로 열린 한국지역정보화학회 세미나에서 서종희 연세대학교 법학전문대학원 교수는 "플랫폼은 국경이 없는 무한 경쟁 시장으로 신규 진입도 자유롭고 경쟁성과 변동성의 폭이 큰 특성상 독과점이 더 어려운 구조"라며 "오히려 국내 제조업 등 신규 경쟁자의 진입이 어려운 특성을 가진 산업군과 비교하면 플랫폼만을 규제하기 위한 법이 수범자(기업)의 관용을 얻기는 어려울 것"이라고 진단했다.'''

In [9]:
laws1 = ner_inference(text1)

In [10]:
laws1

['공',
 '##정',
 '##경',
 '##쟁',
 '_',
 '촉',
 '##진',
 '##법',
 '##제',
 '##내',
 '규',
 '##제',
 '##하',
 '##기',
 '_',
 '위',
 '##한',
 '_',
 '법']

In [11]:
text3 = '''선거법 위반을 제외한 나머지, 즉 공무상비밀누설, 개인정보보호법 위반, 형사사법절차전자화촉진법위반 혐의를 가지고 
집행유예 없는 실형을 선고한 것은 꽤 중한 처분이다. 
게다가 손 검사장은 전과도 없고 현직 검사장 신분이다. 
'''

In [12]:
laws3 = ner_inference(text3)

In [22]:
laws3

['선',
 '##거',
 '##법',
 '개',
 '##인',
 '##정',
 '##보',
 '##보',
 '##호',
 '##법',
 '형',
 '##사',
 '##사',
 '##법',
 '##절',
 '##차',
 '##전',
 '##자',
 '##화',
 '##촉',
 '##진',
 '##법']

# 함수 

In [135]:
def extract_law(text):
    processed_text = text.replace("'", "").replace('"', '')
    laws = ner_inference(processed_text)
    input_list = laws
    output_list = []
    current_word = ""

    for item in input_list:
        if '##' in item:
            current_word += item.replace('##', '')
        else:
            output_list.append(current_word)
            current_word = item

    # Append the last word
    output_list.append(current_word)

    # Remove empty strings from output_list
    output_list = list(filter(None, output_list))

    if '_' in output_list:
        modified_list = [item.replace('_', ' ') if '_' in item else item for item in output_list]
        output_string = ''.join(modified_list)
        output_list = [output_string]
    
    output_list = [item for item in output_list if len(item) > 1 and '법' in item]
    return output_list

In [136]:
print(extract_law(text1))

['공정경쟁 촉진법규제하기 위한 법']


In [137]:
print(extract_law(text2))

['이태원 참사 특별법']


In [138]:
print(extract_law(text3))

['선거법', '개인정보보호법', '형사사법절차전자화촉진법']


In [131]:
text4 = '''서울 동작경찰서는 지난 29일 아동청소년성보호법 위반, 성매매 알선 등 혐의로 A(42)씨를 불구속 송치했다고 31일 밝혔다.'''

In [139]:
print(extract_law(text4))

['아동청소년성보호법']


In [133]:
text5 = '''노후계획도시의 정의를 담은 노후계획도시정비특별법 시행령 제정안이 마련되면서 전국 108개 내외 지역이 법 적용 대상으로 압축됐다.'''

In [140]:
print(extract_law(text5))

['노후계획도시정비특별법']
